<a href="https://colab.research.google.com/github/Berlingrad/MLinCyberSecurityProject/blob/XuDevelopment/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN Back Door Detector

## Preprocessing

###Data Download
Clone the CSAW competition repo and download provided datasets from the Google drive. Downloaded data are moved <code>/content/CSAW-HackML-2020/data</code> directory.

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=1-ay0w_hgTMo6ZNxb7CZhqxWZhkMj1I6Z
!gdown https://drive.google.com/uc?id=1A1CnWy2N1gjzWO6ZEUgiQ-LOAMgKvElL
!gdown https://drive.google.com/uc?id=1ZH7jMGEKhtmI-K0Jw-oKv843jxs-_kkI
!gdown https://drive.google.com/uc?id=1Sw77QLiBn-FuXpPXnas-Gdo3M_aQBD_H

!mv /content/*.h5 /content/CSAW-HackML-2020/data


Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 164, done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 164
Receiving objects: 100% (164/164), 79.56 MiB | 38.30 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Downloading...
From: https://drive.google.com/uc?id=1-ay0w_hgTMo6ZNxb7CZhqxWZhkMj1I6Z
To: /content/clean_test_data.h5
398MB [00:02, 199MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A1CnWy2N1gjzWO6ZEUgiQ-LOAMgKvElL
To: /content/clean_validation_data.h5
716MB [00:04, 175MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZH7jMGEKhtmI-K0Jw-oKv843jxs-_kkI
To: /content/sunglasses_poisoned_data.h5
398MB [00:07, 54.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Sw77QLiBn-FuXpPXnas-Gdo3M_aQBD_H
To: /content/anonymous_1_poisoned_data.h5
637MB [00:17, 37.4MB/s]


In [2]:
import os
import gdown
import numpy as np
import random
import h5py
import glob2
import tempfile
import copy

import keras
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

##check
if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")

     |████████████████████████████████| 174kB 8.9MB/s 


Functions for loading and decompresssing data and models from h5 format. If you added a dataset, simply run load_dataset with the path to the h5

In [3]:
def load_dataset(data_filename, keys=None):
    dataset = {}
    nClass = 1283
    with h5py.File(data_filename, 'r') as hf:
        if keys is None:
            for name in hf:
                dataset[name] = np.array(hf.get(name))
        else:
            for name in keys:
                dataset[name] = np.array(hf.get(name))
                
        dataset['data'] /= 255

        y = np.zeros((dataset['label'].shape[0], nClass))

        for i in range(dataset['label'].shape[0]):
            y[i][int(dataset['label'][i])] = 1

        dataset['data'] = dataset['data'].transpose((0,2,3,1))
        dataset['label'] = y
        x = dataset['data']

    return x, y, dataset

def load_models(model_dir):
    models = []
    names = {}

    for i, net in enumerate(glob2.glob("/content/CSAW-HackML-2020/models/*net.h5")):
        print("loading", net)
        weights = glob2.glob(net.split("_")[0]+"*weights.h5")[0]
       
        m = load_model(net)
        m.load_weights(weights)

        models.append(m)
        
        names[i] = os.path.basename(net).split(".")[0]
    return models, names

###Load Data

In [4]:
SG_poisoned_path = "/content/CSAW-HackML-2020/data/sunglasses_poisoned_data.h5"
xSG, ySG, SG_poisoned = load_dataset(SG_poisoned_path)

clean_validation_path = "/content/CSAW-HackML-2020/data/clean_validation_data.h5"
xValid, yValid, valid_clean = load_dataset(clean_validation_path)

clean_test_path = "/content/CSAW-HackML-2020/data/clean_test_data.h5"
xTest, yTest, test_clean = load_dataset(clean_test_path)

anon_poisoned_path = "/content/CSAW-HackML-2020/data/anonymous_1_poisoned_data.h5"
xAnon, yAnon, anon_poisoned = load_dataset(anon_poisoned_path)


####Mixed final test data
Mix the sunglass poinsoned images with clean test images from test set to create a final validation set. Warning: may cause crash due to RAM shortage.

In [5]:
##TODO
def mix_dataset(A, B):
    nSample = A['data'].shape[0] + B['data'].shape[0] 
    offset = A['data'].shape[0]
    xShape = (nSample, 55, 47, 3)
    yShape = (nSample, 1283+1)
    xOut = np.zeros(xShape)
    yOut = np.zeros(yShape)

    iOut = [i for i in range(nSample)]
    random.shuffle(iOut)

    for i in iOut:
        if i >= offset:

            xOut[i] = B['data'][i-offset]

            if np.argmax(B['label'][i-offset]) != 0:
                yOut[i] = np.append(B['label'][i-offset], [0])

            else:
                yOut[i][-1] = 0
            
        else:
            xOut[i] = A['data'][i]
            if np.argmax(A['label'][i]) != 0:
                yOut[i] = np.append(A['label'][i], [0])
            else:
                yOut[i][-1] = 0

    return xOut, yOut 

In [6]:
##xSGmixed, ySGmixed = mix_dataset(SG_poisoned, test_clean)

###Load Models
Load all Badnet models under <code>/content/CSAW-HackML-2020/models</code> directory. 

In [7]:

model_dir = "/content/CSAW-HackML-2020/models"

models, model_names = load_models(model_dir)

model_names.values()


loading /content/CSAW-HackML-2020/models/anonymous_2_bd_net.h5
loading /content/CSAW-HackML-2020/models/multi_trigger_multi_target_bd_net.h5
loading /content/CSAW-HackML-2020/models/sunglasses_bd_net.h5
loading /content/CSAW-HackML-2020/models/anonymous_1_bd_net.h5


dict_values(['anonymous_2_bd_net', 'multi_trigger_multi_target_bd_net', 'sunglasses_bd_net', 'anonymous_1_bd_net'])

### Model Summary

In [8]:
##TODO
for i, model in enumerate(models):
    print("model ", i, ":")
    model.summary()

     

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
_________________________________________________________________________________

In [9]:
def validate_model(y, y_hat):
    ##compute model accuracy based vectorized output
    n = 0
    for i, v in enumerate(y):
        correct = 1 if np.argmax(v) == np.argmax(y_hat[i]) else 0
        n+= correct 
    return n/(i + 1)
    
    

####Evaluate loaded models on poisoned data.

 Note poisoned data will not involved in the repairing process. We are evaluating it to set baselines, Later we will use the results to validate our method 

In [10]:
for i, m in enumerate(models):
    tf.keras.backend.clear_session()
    predClean = m.predict(xTest)
    predSG = m.predict(xSG)
    predAnon = m.predict(xAnon)
    print("badnet", model_names[i], "accuracy on clean test set: ", validate_model(predClean, yTest))
    print("badnet", model_names[i], "accuracy on sunglass poisoned set: ", validate_model(predSG, ySG))
    print("badnet", model_names[i], "accuracy on anonumous poisoned set: ", validate_model(predAnon, yAnon))

badnet anonymous_2_bd_net accuracy on clean test set:  0.9596258768511302
badnet anonymous_2_bd_net accuracy on sunglass poisoned set:  0.0
badnet anonymous_2_bd_net accuracy on anonumous poisoned set:  0.0
badnet multi_trigger_multi_target_bd_net accuracy on clean test set:  0.9600935307872175
badnet multi_trigger_multi_target_bd_net accuracy on sunglass poisoned set:  0.0
badnet multi_trigger_multi_target_bd_net accuracy on anonumous poisoned set:  0.0
badnet sunglasses_bd_net accuracy on clean test set:  0.9777864380358535
badnet sunglasses_bd_net accuracy on sunglass poisoned set:  0.9999220576773188
badnet sunglasses_bd_net accuracy on anonumous poisoned set:  0.0010717069368667187
badnet anonymous_1_bd_net accuracy on clean test set:  0.9596258768511302
badnet anonymous_1_bd_net accuracy on sunglass poisoned set:  0.0
badnet anonymous_1_bd_net accuracy on anonumous poisoned set:  0.0


###Save pre-repaired results
For later poisoned image detection.

####Anon Poisoned

In [11]:
preds_preprocessed = [] ##test result of the badnets before repair

for i, m in enumerate(models):
    tf.keras.backend.clear_session() 
    pred = m.predict(xAnon)
    preds_preprocessed.append(pred)

test_preprocessed = []
for i, m in enumerate(models):
    tf.keras.backend.clear_session()
    pred = m.predict(xTest)
    test_preprocessed.append(pred)


####test clean

####Load your own data

In [12]:
'''
Path_to_your_data = 
yourX, yourY, yourDataset = load_dataset(Path_to_your_data)
preds_preprocessed = []

preds_preprocessed = [] ##test result of the badnets before repair

for i, m in enumerate(models):
    pred = m.predict(yourX)
    preds_preprocessed.append(pred)

'''

'\nPath_to_your_data = \nyourX, yourY, yourDataset = load_dataset(Path_to_your_data)\npreds_preprocessed = []\n\npreds_preprocessed = [] ##test result of the badnets before repair\n\nfor i, m in enumerate(models):\n    pred = m.predict(yourX)\n    preds_preprocessed.append(pred)\n\n'

##Pruning

Iteratively prune the each base badnet models 3 times with <code>prune_low_magenitue</code> in Keras. The clean Validation set is used for fit. Pruned nets are saved in list <code>pruned_models</code>

In [13]:

pruned_models = []
for i, base_model in enumerate(models):
    for _ in range(3):
        model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)
    
        log_dir = tempfile.mkdtemp()
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
        ]
        model_for_pruning.compile(
          loss=tf.keras.losses.categorical_crossentropy,
          optimizer='adam',
          metrics=['accuracy']
        )
        model_for_pruning.fit(xValid, yValid,
                              callbacks=callbacks,
                              epochs=2
                              )

    pruned_models.append(model_for_pruning)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2
361/361 [==============================] - 9s 17ms/step - loss: 0.4658 - accuracy: 0.8981
Epoch 2/2
361/361 [==============================] - 6s 17ms/step - loss: 0.0844 - accuracy: 0.9755
Epoch 1/2
361/361 [==============================] - 9s 17ms/step - loss: 0.1245 - accuracy: 0.9638
Epoch 2/2
361/361 [==============================] - 6s 16ms/step - loss: 0.0551 - accuracy: 0.9842
Epoch 1/2
361/361 [==============================] - 9s 18ms/step - loss: 0.0928 - accuracy: 0.9749
Epoch 2/2
361/361 [==============================] - 6s 17ms/step - loss: 0.0550 - accuracy: 0.9841
Epoch 1/2
361/361 [==============================] - 9s 17ms/step - loss: 0.4496 - accuracy: 0.9025
Epoch 2/2
361/361 [==============================] - 6s 16ms/step - loss: 0.0757 - accuracy: 0.9815
Epoch 1/2
361/361 [==============================] - 9s 17ms/step - loss: 0.1188 - accuracy: 0.9660
Epoch 2/2
361/361 [==============================] - 6s 16ms/step - loss: 0.0695 - accuracy: 0.9791


In [14]:
for i, model in enumerate(pruned_models):
    print("model ", i, ":")
    model.summary()

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
_________________________________________________________________________________

##Fine-tuning
Fine-tuning the pruned models with <code>1e-5</code> learning rate in 10 epochs. The clean validation set is used in the training process. Repaired nets are saved in list <code>repaired_models</code>

In [15]:
repaired_models = []

for i, m in enumerate(pruned_models):

    model = copy.copy(m)

    print("Fine-tuning model", model_names[i], ": ")
    model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy']
              )
    model.fit(xValid, yValid,
          epochs=5
          )
    repaired_models.append(model)

Fine-tuning model anonymous_2_bd_net : 
Epoch 1/5
361/361 [==============================] - 8s 13ms/step - loss: 0.0360 - accuracy: 0.9917
Epoch 2/5
361/361 [==============================] - 5s 13ms/step - loss: 0.0179 - accuracy: 0.9952
Epoch 3/5
361/361 [==============================] - 5s 14ms/step - loss: 0.0134 - accuracy: 0.9965
Epoch 4/5
361/361 [==============================] - 5s 15ms/step - loss: 0.0131 - accuracy: 0.9964
Epoch 5/5
361/361 [==============================] - 5s 14ms/step - loss: 0.0059 - accuracy: 0.9983
Fine-tuning model multi_trigger_multi_target_bd_net : 
Epoch 1/5
361/361 [==============================] - 7s 13ms/step - loss: 0.0257 - accuracy: 0.9919
Epoch 2/5
361/361 [==============================] - 5s 14ms/step - loss: 0.0186 - accuracy: 0.9947
Epoch 3/5
361/361 [==============================] - 5s 13ms/step - loss: 0.0155 - accuracy: 0.9956
Epoch 4/5
361/361 [==============================] - 5s 13ms/step - loss: 0.0100 - accuracy: 0.9972
Epoch

##Validation
Demonstrate the effectiveness of our results by run clean_test, Sunglass_poisoned, and anonymous_poisoned data on all repaired nets. 

In [16]:
##Intermediate quick evalution using built-in Keras functions and the poisoned set

for i, m in enumerate(repaired_models):
    print("Repaired", model_names[i], "accuracy on clean test set: ")
    m.evaluate(xTest, yTest) 
    print("Repaired", model_names[i], "accuracy on sunglass poisoned set: ")
    m.evaluate(xSG, ySG)
    
    print("Repaired", model_names[i], "accuracy on anonymous poisoned set: ")
    m.evaluate(xAnon, yAnon) 

Repaired anonymous_2_bd_net accuracy on clean test set: 
401/401 [==============================] - 2s 4ms/step - loss: 0.5130 - accuracy: 0.9242
Repaired anonymous_2_bd_net accuracy on sunglass poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 64.1384 - accuracy: 0.0000e+00
Repaired anonymous_2_bd_net accuracy on anonumous poisoned set: 
321/321 [==============================] - 1s 4ms/step - loss: 41.2215 - accuracy: 8.7685e-04
Repaired multi_trigger_multi_target_bd_net accuracy on clean test set: 
401/401 [==============================] - 2s 4ms/step - loss: 0.4990 - accuracy: 0.9245
Repaired multi_trigger_multi_target_bd_net accuracy on sunglass poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 44.8559 - accuracy: 0.0000e+00
Repaired multi_trigger_multi_target_bd_net accuracy on anonumous poisoned set: 
321/321 [==============================] - 1s 4ms/step - loss: 42.2676 - accuracy: 0.0011
Repaired sunglasses_bd_net accura

##Final output
Compare preprocessed results from the bad net and results from repaired nets to output results. The differnt predictions between bad nets and repaired nets are classifed as N+1 class. Here we use <code>anonymous_poisoned</code> as our input. 

In [17]:
def poisoned_detection(afterPreds, prePreds):
    out = []
    for i in range(len(afterPreds)):
        newDimension = list(afterPreds[i].shape)
        newDimension[1] += 1
        newArray = np.zeros(newDimension)
        for j in range(afterPreds[i].shape[0]):
            a = np.argmax(afterPreds[i][j]) 
            b = np.argmax(prePreds[i][j])
            
            if a == b:
                newArray[j][a] = 1
            else:
                newArray[j][-1] = 1
        out.append(newArray)
    return out

In [18]:
preds_repaired = [] ##prediction on poisoned data
test_repaired = [] ##prediction on clean test data

for i, m in enumerate(repaired_models):
    pred = m.predict(xAnon)
    pred_t = m.predict(xTest)
    preds_repaired.append(pred)
    test_repaired.append(pred_t)
 

In [19]:
test_results = poisoned_detection(test_repaired, test_preprocessed)

final_results = poisoned_detection(preds_repaired, preds_preprocessed)

###Poisoned image detection validation
Evaluate effectiveness of our poisoned detection method on mixed <code>anonymous_poisoned</code> and <code>test_clean</code>

In [21]:
for i in range(len(final_results)):
    tp = 0
    fn = 0
    tn = 0
    t=0
    
    p = final_results[i]
    c = test_results[i]

    n = p.shape[0]
    m = c.shape[0]

    for j in range(n): 

        trueLabel = np.argmax(yAnon[j])
        predLabel = np.argmax(p[j])

        if predLabel == 1283:
            t+=1
            if trueLabel == 0:
                tp += 1
        else:
            if trueLabel != 0:
                tn += 1

        if trueLabel == 0:
            if predLabel != 1283:
                fn += 1

    for j in range(m):

        trueLabel = np.argmax(yTest[j])
        predLabel = np.argmax(c[j])

        if predLabel == 1283:
            t+=1
            if trueLabel == 0:
                tp += 1
        else:
            if trueLabel != 0:
                tn += 1
        if trueLabel == 0:
            if predLabel != 1283:
                fn += 1

    print("Repaired", model_names[i], "poisoned detection:")
    
    acc = (tp + tn)/(n + m)
    tpr = tp/(fn+tp)
    tnr = 1 - (fn/(fn+tp))
    print("Acc: ", acc)
    print("TPR: ", tpr)
    ##print("TNR: ", tnr)

    print("-"*10)


Repaired anonymous_2_bd_net poisoned detection:
Acc:  0.8906209405040271
TPR:  0.8742456686782168
----------
Repaired multi_trigger_multi_target_bd_net poisoned detection:
Acc:  0.8324673075257643
TPR:  0.7390500291999221
----------
Repaired sunglasses_bd_net poisoned detection:
Acc:  0.5508790161946826
TPR:  0.12156900914930893
----------
Repaired anonymous_1_bd_net poisoned detection:
Acc:  0.9106261366588725
TPR:  0.9232042047887872
----------
